In [7]:
import string
import nltk
import re
import csv
import ast

from re import compile
#for sorting dictionary
import operator
from nltk.corpus import stopwords
from nltk.collocations import *

def tokenize(text):
    tokens=[]
    # write your code here
    stop_words = stopwords.words('english')
    text = text.lower()
    #print(text)
    #pattern = r'\w'
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip(string.punctuation) for token in tokens]
    
    tokens = [token for token in tokens if re.match("^[A-Za-z_-]*$", token)]
    #print(tokens)
    # to remove punctuations from begging and starting of the tokens 
    
    # now removing extra empty characters from tokens
    tokens = [token.strip() for token in tokens if token.strip()!='']
    tokens = [token for token in tokens if len(token)>1]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [8]:
import string
import nltk
import re
import csv
import ast
#for initial data filtering 
import preprocessing
from re import compile
#for sorting dictionary
import operator
from nltk.corpus import stopwords
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()


def dateToSum(filename):
    temp = 0
    with open(filename, "r",encoding="ISO-8859-1") as f:
        reader = csv.reader(f, delimiter=',')

        rows = [row[7] for row in reader]
        tot = len(rows)
        # print(tot)
        for rro in rows:
            if len(rro) > 0:
                exp_len = ast.literal_eval(rro)
                # print(exp_len)
                for t in exp_len:
                    tokens = nltk.word_tokenize(t)
                    for i in range(len(tokens)):
                        if tokens[i] == "NA":
                            tokens[i] = ""
                        elif tokens[i] == "Present":
                            tokens[i] = "2018"
                    rex = compile('[^0-9]')
                    filteredData = [x for x in tokens if not rex.match(x)]

                    if len(filteredData) == 2:
                        temp = (int(filteredData[1]) - int(filteredData[0])) + temp

        print("\n\nAverage Work Experience : ")
        print(round(temp/tot))


def performance_evaluate(input_file):
    trigrams_list =[]
    bigrams_list=[]
    # write your code here
    with open(input_file, "r",encoding="ISO-8859-1") as f:
        reader=csv.reader(f, delimiter=',')
        # row[1] or second column is header of resume which states the current job position.
        # Although we did data scraping for particular job positions, current job positions varied a lot
        # in initial analysis phase- so even after scraping resumes for one particular position, we are making sure the
        # actual job postion for which analysis will be done.
        rows=[(row[1], row[5], row[4], row[10], row[3]) for row in reader]
        
   
    #print(row_len)
    for i in rows:
        token_list = tokenize(i[0])
        tmp_list = list(nltk.trigrams(token_list))
        tmp_big_list = list(nltk.bigrams(token_list))
        #print(tmp_big_list)
        for j in tmp_list:
            trigrams_list.append(j)
        for l in tmp_big_list:
            bigrams_list.append(l)
    #print(bigrams_list)      
    
    #Here for job titles we cannot use NLTK's trigram collocation finder as it will filter out some important details.
    #so finding it manually.
    trigram_freq={}
    bigram_freq={}
    
    for k in trigrams_list:
        if k in trigram_freq:
            trigram_freq[k]+=1
        else:
            trigram_freq[k]=1
            
    for k in bigrams_list:
        if k in bigram_freq:
            bigram_freq[k]+=1
        else:
            bigram_freq[k]=1
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(trigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(sorted_freq)
    #print("Sorted by frequency - trigrams from heading\n\nSorted by Frequency Bigrams from heading")
    #for bigrams
    bi_sorted_freq = sorted(bigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(bi_sorted_freq)
    
    # now the top most value is what we are looking for. and all our analysis will be for that position.
    desired_position = sorted_freq[0][0]
    print("\n\nThis analysis is for ",sorted_freq[0][0])
    print("\n##############################################\n")
    #--------------------------------------------
    # now for 5th column i.e. job titles.
    #--------------------------------------------
    exp_trigrams_list =[]
    exp_bigrams_list=[]
    for i in rows:
        #second param is job exp
        exp_list = tokenize(i[1])
        #print(exp_list[0])
        tmp_list = list(nltk.trigrams(exp_list))
        tmp_big_list = list(nltk.bigrams(exp_list))
        
        for j in tmp_list:
            exp_trigrams_list.append(j)
        for l in tmp_big_list:
            exp_bigrams_list.append(l)
    
    exp_trigram_freq={}
    exp_bigram_freq={}
    
    for k in exp_trigrams_list:
        if k in exp_trigram_freq:
            exp_trigram_freq[k]+=1
        else:
            exp_trigram_freq[k]=1
       
    for k in exp_bigrams_list:
        if k in exp_bigram_freq:
            exp_bigram_freq[k]+=1
        else:
            exp_bigram_freq[k]=1
      
    
    exp_sorted_freq = sorted(exp_trigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(exp_sorted_freq)
    print("'input resume' can have more chances to become  ",desired_position," if it has following job experience terms\n")
    
    for i in range(0,10):
        print(exp_sorted_freq[i][0])
    print("\n\nOR----####----####----####\n\n")
    #for bigrams
    exp_bi_sorted_freq = sorted(exp_bigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(exp_bi_sorted_freq)
    for i in range(0,10):
        print(exp_bi_sorted_freq[i][0])
    
    
    print("\n##############################################\n")
    #--------------------------------------------
    # now for 4th column i.e. skills.
    #--------------------------------------------
    skills_list = []
    for i in rows:
        token_list = tokenize(i[2])
        
        for j in token_list:
            skills_list.append(j)
        
    #print(len(skills_list))
    skill_dict={}
    for k in skills_list:
        if k in skill_dict:
            skill_dict[k]+=1
        else:
            skill_dict[k]=1
            
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(skill_dict.items(), key=operator.itemgetter(1),reverse=True)
    print("\n\nThese are preferred skills for the position.\n")
    custom = ["years","year","na","less","than","and"]
    for i in range(0,35):
        if(sorted_freq[i][0] not in custom):
            print(sorted_freq[i][0], sorted_freq[i][1])
    
    print("\n##############################################\n")
    #--------------------------------------------
    # now for 10th column i.e. skills.
    #--------------------------------------------
    skills_list = []
    trigrams_list = []
    for i in rows:
        token_list = tokenize(i[3])
        tmp_list = list(nltk.trigrams(token_list))
        for j in tmp_list:
            trigrams_list.append(j)
        for k in token_list:
            skills_list.append(k)
        
    
    skill_dict={}
    trigram_freq = {}
    for l in trigrams_list:
        if l in trigram_freq:
            trigram_freq[l]+=1
        else:
            trigram_freq[l]=1
    for k in skills_list:
        if k in skill_dict:
            skill_dict[k]+=1
        else:
            skill_dict[k]=1
            
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(skill_dict.items(), key=operator.itemgetter(1),reverse=True)
    tri_education = sorted(trigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    print("\n\nThese are preferred Education details for the position.\n")
    for j in range(0,8):
        print(tri_education[j])
    
    custom = ["years","year","na","less","than","and","in","of","data","technology"]
    for i in range(0,8):
        if(sorted_freq[i][0] not in custom):
            print(sorted_freq[i])
    
    
    print("\n##############################################\n")
    print("\nWork authorization requirement for this position in USA\n")
    #--------------------------------------------
    # now for 10th column i.e. skills.
    #--------------------------------------------
    auth_list=[]
    na = ["NA","na"]
    for i in rows:
        auth_list.append(i[4])
    auth_dict = {}
    for i in auth_list:
        if i in auth_dict:
            auth_dict[i]+=1
        else:
            auth_dict[i]=1
    sort_auth =  sorted(auth_dict.items(), key=operator.itemgetter(1),reverse=True)
    for j in range(0,3):
        if (sort_auth[j][0] not in na):
            print(sort_auth[j][0])
            
            
def description_analysis(input_file):
    trigrams_list =[]
    bigrams_list=[]
    # write your code here
    with open(input_file, "r",encoding="ISO-8859-1") as f:
        reader=csv.reader(f, delimiter=',')
        rows=[(row[8]) for row in reader]
    text=". ".join(rows).lower()
    
    #print(row_len)
    for i in rows:
        token_list = tokenize(i)
        tmp_list = list(nltk.trigrams(token_list))
        tmp_big_list = list(nltk.bigrams(token_list))
        #print(tmp_big_list)
        for j in tmp_list:
            trigrams_list.append(j)
        for l in tmp_big_list:
            bigrams_list.append(l)
    #print(bigrams_list)      
    finder = BigramCollocationFinder.from_words(\
        nltk.word_tokenize(text))
    
    finder.apply_freq_filter(5)
    print(finder.nbest(bigram_measures.pmi, 10))
    
    
if __name__ == "__main__":
    #this is for job title analysis
    performance_evaluate("indeed_scraped_data_science.csv")
    #this is for calculating avg job experience for data science
    print("\n##############################################\n")
    dateToSum("indeed_scraped_data_science.csv")
    print("\n##############################################\n")
    print("\n\nBelow are the preferred job experience terms for the given position.\n")
    #this function will analyze the job description of all data science resumes.
    description_analysis("indeed_scraped_data_science.csv")
  
    



This analysis is for  ('senior', 'software', 'engineer')

##############################################

'input resume' can have more chances to become   ('senior', 'software', 'engineer')  if it has following job experience terms

('senior', 'software', 'engineer')
('data', 'scientist', 'data')
('software', 'engineer', 'senior')
('scientist', 'data', 'scientist')
('engineer', 'senior', 'software')
('sr', 'software', 'engineer')
('software', 'engineer', 'software')
('data', 'scientist', 'intern')
('engineer', 'software', 'engineer')
('vice', 'president', 'vice')


OR----####----####----####


('software', 'engineer')
('data', 'scientist')
('senior', 'software')
('vice', 'president')
('scientist', 'data')
('engineer', 'senior')
('data', 'analyst')
('sr', 'software')
('project', 'lead')
('engineer', 'software')

##############################################



KeyboardInterrupt: 